In [2]:
import s3fs
from glob import glob`
import xarray as xr

## Specify the S3 bucket where the NPS assets are

In [3]:
bucket = 's3://npwbanalres'

## Create a connection to the S3 bucket and list all of the assets 

In [4]:
s3 = s3fs.S3FileSystem(anon=False)

List all files in bucket

In [5]:
#s3.glob(f'{bucket}/*.nc4')

In [6]:
years = list(set(int(v.split('_')[3].split('.')[0]) for v in s3.glob(f'{bucket}/*.nc4') if 'monthly' not in v))
print(f'Number of Years: {len(years)}')
print(f'Year Range: {min(years)} to {max(years)}')

Number of Years: 42
Year Range: 1980 to 2021


## List the variables for both daily and monthly assets

Daily variable names

In [28]:
wb_variables = list(set(v.split('_')[-1].split('.')[0] for v in s3.glob(f'{bucket}/*.nc4') if 'monthly' not in v))
wb_variables.sort()

In [29]:
wb_variables

['accumswe', 'aet', 'deficit', 'pet', 'rain', 'runoff', 'soilwater']

Monthly variable names

In [30]:
wb_m_variables = list(set(f"{v.split('_')[-2]}_{v.split('_')[-1].split('.')[0]}" for v in s3.glob(f'{bucket}/*.nc4') if 'monthly' in v))
wb_m_variables.sort()

In [31]:
wb_m_variables

['accumswe_monthly',
 'aet_monthly',
 'deficit_monthly',
 'pet_monthly',
 'rain_monthly',
 'runoff_monthly',
 'soilwater_monthly']

## Connect to an NPS asset in S3

In [32]:
var = wb_m_variables[1]
var

'aet_monthly'

In [33]:
urls = s3.glob(f'{bucket}/*{var}.nc4')
urls

['npwbanalres/v_1_5_1980_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1981_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1982_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1983_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1984_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1985_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1986_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1987_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1988_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1989_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1990_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1991_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1992_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1993_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1994_gridmet_historical_aet_monthly.nc4',
 'npwbanalres/v_1_5_1995_gridmet_historical_aet_monthly.nc4',
 'npwban

### Read in a single asset

In [35]:
url = urls[-10]    # -10 is 2012
url

'npwbanalres/v_1_5_2012_gridmet_historical_aet_monthly.nc4'

In [36]:
s3_file_obj = s3.open(url, mode='rb')

In [37]:
%%time
xr_ds = xr.open_dataset(s3_file_obj, chunks='auto', engine='h5netcdf', mask_and_scale=True)
xr_ds

CPU times: user 516 ms, sys: 103 ms, total: 618 ms
Wall time: 2.06 s


<xarray.Dataset>
Dimensions:                  (time: 12, tbnds: 2, x: 4700, y: 3300)
Coordinates:
  * time                     (time) datetime64[ns] 2012-01-16 ... 2012-12-15
  * x                        (x) float32 -2.06e+06 -2.059e+06 ... 2.639e+06
  * y                        (y) float32 1.384e+06 1.383e+06 ... -1.915e+06
    lat                      (y, x) float32 dask.array<chunksize=(3300, 4700), meta=np.ndarray>
    lon                      (y, x) float32 dask.array<chunksize=(3300, 4700), meta=np.ndarray>
Dimensions without coordinates: tbnds
Data variables:
    time_bnds                (time, tbnds) datetime64[ns] dask.array<chunksize=(12, 2), meta=np.ndarray>
    timeunit                 (time) int16 dask.array<chunksize=(12,), meta=np.ndarray>
    lambert_conformal_conic  int16 ...
    aet                      (time, y, x) float32 dask.array<chunksize=(6, 1980, 2822), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    NCO:          netCDF Operators version 5.0.4 (Homepage = http://nco.sf.ne...
    history:      Fri Jan 27 23:29:35 2023: ncatted -O -a units,time,o,c,days...

`If` you have x and y coordinates in the source proj, enter them below

In [ ]:
x,y = 1787841,-1531176 

`Else`, convert geographic coordinates to source proj to `sel` against the x and y coordinates

In [42]:
import cartopy.crs as ccrs

In [43]:
inLat = 44.04858398
inLon = -103.6933594

In [44]:
# Example - your x and y coordinates are in a Lambert Conformal projection
data_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=42.5, false_easting=0.0, false_northing=0.0, standard_parallels=(25,60))

In [45]:
# Transform the point - src_crs is always Plate Carree for lat/lon grid
x, y = data_crs.transform_point(inLon, inLat, src_crs=ccrs.PlateCarree())

In [46]:
x

-282135.9863446089

In [47]:
y

170287.85426675095

Now you can select data using the x and y

In [50]:
xr_ds.sel(x=x, y=y, method='nearest').sel(time='2012').aet.values

array([ 23.2     ,  29.9     ,  40.100002,  43.      ,  91.9     ,
       114.      , 162.1     , 161.5     , 133.2     , 104.      ,
        36.100002,  44.4     ], dtype=float32)

In [38]:
xr_ds.time.values

array(['2012-01-16T00:00:00.000000000', '2012-02-15T00:00:00.000000000',
       '2012-03-15T00:00:00.000000000', '2012-04-15T00:00:00.000000000',
       '2012-05-15T00:00:00.000000000', '2012-06-15T00:00:00.000000000',
       '2012-07-15T00:00:00.000000000', '2012-08-15T00:00:00.000000000',
       '2012-09-15T00:00:00.000000000', '2012-10-15T00:00:00.000000000',
       '2012-11-15T00:00:00.000000000', '2012-12-15T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [ ]:
TODO: Use selection method above against multiple files

### Read in multiple assets

In [13]:
%%time
ds_all = []
for url in urls:
    #s3_file_obj = s3.open(url, mode='rb')
    ds_all.append(xr.open_dataset(s3.open(url, mode='rb'), chunks='auto', engine='h5netcdf'))

CPU times: user 16.5 s, sys: 3.75 s, total: 20.3 s
Wall time: 1min 19s


In [14]:
%%time
ds_ts = xr.concat(ds_all, dim='time')
ds_ts

CPU times: user 31 s, sys: 5.06 s, total: 36 s
Wall time: 1min 57s


<xarray.Dataset>
Dimensions:                  (time: 15341, tbnds: 2, x: 4700, y: 3300)
Coordinates:
  * time                     (time) datetime64[ns] 1980-01-01T12:00:00 ... 20...
  * x                        (x) float32 -2.06e+06 -2.059e+06 ... 2.639e+06
  * y                        (y) float32 1.384e+06 1.383e+06 ... -1.915e+06
    lat                      (y, x) float32 51.78 51.79 51.79 ... 21.12 21.12
    lon                      (y, x) float32 -131.7 -131.7 ... -74.77 -74.76
Dimensions without coordinates: tbnds
Data variables:
    time_bnds                (time, tbnds) datetime64[ns] dask.array<chunksize=(366, 2), meta=np.ndarray>
    timeunit                 (time) int16 dask.array<chunksize=(366,), meta=np.ndarray>
    lambert_conformal_conic  (time) int16 -32767 -32767 -32767 ... -32767 -32767
    accumswe                 (time, y, x) float32 dask.array<chunksize=(66, 596, 850), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    NCO:          netCDF Operators version 5.0.4 (Homepage = http://nco.sf.ne...
    history:      Fri Dec 30 04:28:32 2022: ncatted -a missing_value,accumswe...

---

## Testing

In [15]:
# for v in wb_variables:
#     print(v)
#     urls = s3.glob(f'{bucket}/*{v}.nc4')
#     #print(urls)
#     ly_noly = [0,1,2,3,4,5,6,7,8,9,10]
#     for y in ly_noly:
#         #print(y)
#         url = urls[y]
#         print(url)
#         try:
#             s3_file_obj = s3.open(url, mode='rb')
#             xr_ds = xr.open_dataset(s3_file_obj, chunks='auto', engine='h5netcdf')
#             print(xr_ds.dims)
#             print(xr_ds[v].attrs)
#         except:
#             print(f'FAILED: {url}')